# Data Aggregation and Group Operations

## GroupBy mechanics

### Iterating over groups

### Selecting a column or subset of columns

## Data aggregation

### Column-wise and multiple function application

## Group-wise operations and transformations

### Apply: General split-apply-combine

#### Suppressing the group keys

### Quantile and bucket analysis

### Example: Filling missing values with group-specific values

## Pivot tables and Cross-tabulation